In [1]:
import os
import numpy as np

from hmog import HmogHelper
from tqdm import tqdm

dataset_path = 'hmog_public_dataset/'

In [10]:
subjects = HmogHelper.list_folder(dataset_path)

session_maps = {}

for subject in subjects:
    ww_subject_sessions = HmogHelper.filter_user_sessions_by_type(os.path.join(dataset_path, subject), HmogHelper.WW_SESSION_TYPES)
    
    session_maps[subject] = []
    for session in ww_subject_sessions:
        session_maps[subject].append(HmogHelper.read_person_session(os.path.join(dataset_path, subject, session)))
        HmogHelper.preprocess_session_data(session_maps[subject][-1])

In [16]:
session_hmog_matrices = {}

for subject in tqdm(session_maps):
    session_hmog_matrices[subject] = []

    sessions = session_maps[subject]

    assert len(sessions) > 2

    v1 = HmogHelper.extract_hmog_features(sessions[0], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)
    v2 = HmogHelper.extract_hmog_features(sessions[1], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)

    train_hmog_vector = np.vstack((v1, v2))
    test_hmog_vector = HmogHelper.extract_hmog_features(sessions[2], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)

    session_hmog_matrices[subject].append(train_hmog_vector)
    session_hmog_matrices[subject].append(test_hmog_vector)

# std_deviations, session_hmog_vector = calc_std_and_scale(session_hmog_vector)

 18%|█▊        | 18/100 [04:02<19:52, 14.54s/it]/Users/maves/Projects/Personal/Thesis/hmog_models/ml/hmog/HmogHelper.py:207: RuntimeWarning: divide by zero encountered in scalar divide
  d_max_to_avg = (t_after_center - t_max) / (avg100msAfter - max_during)
100%|██████████| 100/100 [26:41<00:00, 16.01s/it]


In [17]:
import pickle

# hmog_dataset_file = open('hmog_dataset', 'wb')
# pickle.dump(session_hmog_matrices, hmog_dataset_file)
# hmog_dataset_file.close()

In [3]:
import pickle

test_file = open('hmog_dataset', 'rb')
session_hmog_matrices = pickle.load(test_file)
test_file.close()

# Single subject, out-of-session tests

Let's test how the model recognizes the owner in a separate session.

First two WW sessions are used to fit the model, and the third one is used to conduct the test.

In [7]:
from sklearn.svm import OneClassSVM

true_pos_ratio_sum = 0
count = len(session_hmog_matrices)

for subject in session_hmog_matrices:
    std_deviations, session_hmog_vector = HmogHelper.calc_std_and_scale(session_hmog_matrices[subject][0])
    test_hmog_vector = HmogHelper.scale_by(session_hmog_matrices[subject][1], std_deviations)

    if test_hmog_vector.shape[0] == 0:
        count -= 1
        continue

    svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01).fit(session_hmog_vector)
    true_pos_ratio_sum += HmogHelper.test_hmog(test_hmog_vector, 1, svm)

print(f'Out-of-session whole dataset average true positive ratio: {true_pos_ratio_sum / count}')

Out-of-session whole dataset average true positive ratio: 0.8459135339934246
